# Device

In this notebook you will:

* Encapsulate multiple Signals in a Device

Recommend Prerequisites:

* [Hello Python and Jupyter](./Hello%20Python%20and%20Jupyter.ipynb)
* [Epics Signal](./Epics%20Signal.ipynb)

## Simulated Hardware
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [ ]:
!supervisorctl -c supervisor/supervisord.conf status

## Devices are a heirarchy

A `Device` is a hierarchy composed of Signals and other Devices. The components of a Device can be introspected by layers above ophyd and may be decomposed to, ultimately, the underlying Signals.

In [ ]:
from ophyd import EpicsSignal, EpicsSignalRO

x = EpicsSignal('random_walk:x')
dt = EpicsSignal('random_walk:dt')

It would be convenient if we could read these as a unit, instead of `x.read(); dt.read()`.

In [ ]:
from ophyd import Device, Component

class RandomWalk(Device):
    x = Component(EpicsSignalRO, ':x')
    dt = Component(EpicsSignal, ':dt')
    
random_walk = RandomWalk('random_walk', name='random_walk')
random_walk

The `read()` and `describe()` methods walk the hierarchy.

In [ ]:
random_walk.read()

In [ ]:
random_walk.x.read()

In [ ]:
random_walk.dt.read()

In [ ]:
random_walk.describe()

A Device embodies a certain "layout" of components. We can have multiple Devices with different PV prefixes but the same layout.

In [ ]:
another_random_walk = RandomWalk('another_random_walk', name='another_random_walk')

In [ ]:
another_random_walk.read()

A Device can be made of subdevices.

In [ ]:
class RandomWalks(Device):
    a = Component(RandomWalk, 'random_walk')
    b = Component(RandomWalk, 'another_random_walk')
    
random_walks = RandomWalks('', name='random_walks')
random_walks

In [ ]:
random_walks.read()

In [ ]:
random_walks.a.read()

In [ ]:
random_walks.a.x.read()

## Adding a set method to `Device`

Sometimes, setting a value to a Signal and knowing when it is "done" involves just one PV:

In [ ]:
status = random_walks.a.dt.set(2)

In other cases it involves coordination across multiple PVs, such as a setpoint PV nd a readback PV, or a setpoint PV and a "done" PV. For those cases, we define a `set` method on the Device to manage the coordination across multiple Signals.

In [ ]:
from ophyd import DeviceStatus

class Decay(Device):
    """
    A device with a setpoint and readback that decays exponentially toward the setpoint.
    """
    readback = Component(EpicsSignalRO, ':I')
    setpoint = Component(EpicsSignal, ':SP')
    
    def set(self, setpoint):
        """
        Set the setpoint and return a Status object that monitors the readback.
        """
        status = DeviceStatus(self)
        
        # Wire up a callback that will mark the status object as finished
        # when the readback approaches within some tolerance of the setpoint.
        def callback(old_value, value, **kwargs):
            TOLERANCE = 1  # hard-coded; we'll make this configurable later on...
            if abs(value - setpoint) < TOLERANCE:
                status._finished()
            
        self.readback.subscribe(callback)
        
        # Now 'put' the value.
        self.setpoint.put(setpoint)
        
        # And return the Status object, which the caller can use to
        # tell when the action is complete.
        return status
        
    
decay = Decay('decay', name='decay')
decay

In [ ]:
decay.read()

In [ ]:
status = decay.set(115)

We can watch for completion either by registering a callback:

In [ ]:
def callback():
    print("DONE!")
    
status.add_callback(callback)

or by polling:

In [ ]:
status = decay.set(120)

import time
while not status.done:
    time.sleep(0.01)  # Make sure to sleep to avoid pinning CPU.
print("DONE!")

## Adding a `trigger` method to `Device`

Like `Device.set`, `Device.trigger` can coordinate across multiple PVs to trigger and detector and tell when it is done triggering.

When a bluesky plan obtains a reading from some `device` it typically:

* Calls `device.trigger()` and receives back a status object
* Waits for that status object to complete (while potentially doing other things, like triggering other detectors in parallel)
* Calls `device.read()`

In [ ]:
class TriggeredDetector(Device):
    """
    A detector that requires triggering
    """
    gain = Component(EpicsSignal, ':gain')
    exposure_time = Component(EpicsSignal, ':exposure_time')
    reading = Component(EpicsSignalRO, ':reading')
    acquire = Component(EpicsSignal, ':acquire')
    enabled = Component(EpicsSignal, ':enabled')

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = DeviceStatus(self)
        
        # Wire up a callback that will mark the status object as finished
        # when we see the state flip from "acquiring" to "not acquiring"---
        # that is, a negative edge.
        def callback(old_value, value, **kwargs):
            if old_value == 1 and value == 0:
                status._finished()
            
        self.readback.subscribe(callback)
        
        # Now 'put' 1 to the acquire signal.
        self.acquire.put(1)        

        # And return the Status object, which the caller can use to
        # tell when the action is complete.
        return status
    
triggered_detector = TriggeredDetector('trigger_with_pc', name='triggered_detector')

In [ ]:
status = triggered_detector.trigger()

This `status` object is exactly the same as the one we got from `set()`. We can check completion by registering a callback or polling.

In [ ]:
def callback():
    print("ACQUISITION COMPLETE")

status.add_callback(callback)

In [ ]:
while not status.done:
    time.sleep(0.01)
print("ACQUISITION COMPLETE!")

### Using put-completion

In [ ]:
class TriggeredDetector(Device):
    gain = Component(EpicsSignal, ':gain')
    exposure_time = Component(EpicsSignal, ':exposure_time')
    reading = Component(EpicsSignalRO, ':reading')
    acquire = Component(EpicsSignal, ':acquire')
    enabled = Component(EpicsSignal, ':enabled')

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = self.acquire.set(1)
        return status
    
triggered_detector = TriggeredDetector('trigger_with_pc', name='triggered_detector')

In [ ]:
status = triggered_detector.trigger()

## Status objects

Status objects are like rich Futures. They know whether they are `done`, whether their action finished in `success` or not, and they hold a reference to the `device` that they came from, which can be useful to debugging failures. The `status.watch()` may be used to subscribe to incremental progress updates and is used by bluesky to display progress bars during sets and triggers.

In [ ]:
status

In [ ]:
status.success

In [ ]:
status.device

## Sorting components into "kinds"

In [ ]:
triggered_detector.read()

In [ ]:
class TriggeredDetector(Device):
    """
    A detector that requires triggering
    """
    gain = Component(EpicsSignal, ':gain', kind='CONFIG')
    exposure_time = Component(EpicsSignal, ':exposure_time', kind='CONFIG')
    reading = Component(EpicsSignalRO, ':reading', kind='NORMAL')
    acquire = Component(EpicsSignal, ':acquire', kind='OMITTED')
    enabled = Component(EpicsSignal, ':enabled', kind='OMITTED')

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = self.acquire.set(1)
        return status
    
triggered_detector = TriggeredDetector('trigger_with_pc', name='triggered_detector')

In [ ]:
triggered_detector.read()

In [ ]:
triggered_detector.read_configuration()

In [ ]:
triggered_detector.reading.kind

In [ ]:
triggered_detector.reading.kind = 'HINTED'
triggered_detector.reading.kind

In [ ]:
triggered_detector.hints

## Staging and unstaging

Above we said that when bluesky obtains a reading from some `device` it typically:

* Calls `device.trigger()` and receives back a status object
* Waits for that status object to complete (while potentially doing other things, like triggering other detectors in parallel)
* Calls `device.read()`

If it obtains multiple readings in sequence, it repeats this trigger/wait/read cycle. At 

In [ ]:
from ophyd import set_and_wait

class TriggeredDetector(Device):
    """
    A detector that requires triggering
    """
    gain = Component(EpicsSignal, ':gain', kind='CONFIG')
    exposure_time = Component(EpicsSignal, ':exposure_time', kind='CONFIG')
    reading = Component(EpicsSignalRO, ':reading', kind='NORMAL')
    acquire = Component(EpicsSignal, ':acquire', kind='OMITTED')
    enabled = Component(EpicsSignal, ':enabled', kind='OMITTED')

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = self.acquire.set(1)
        return status
    
    def stage(self):
        self.initial_enabled_state = self.enabled.get()
        set_and_wait(self.enabled, 1)
        return super().stage()
    
    def unstage(self):
        ret =  super().unstage()
        set_and_wait(self.enabled, self.initial_enabled_state)
        return ret
    
triggered_detector = TriggeredDetector('trigger_with_pc', name='triggered_detector')

In [ ]:
triggered_detector.enabled.put(0)
triggered_detector.enabled.get()

In [ ]:
triggered_detector.stage()

In [ ]:
triggered_detector.enabled.get()

In [ ]:
status = triggered_detector.trigger()

In [ ]:
while not status.done:
    time.sleep(0.01)

In [ ]:
triggered_detector.read()

In [ ]:
triggered_detector.unstage()

In [ ]:
triggered_detector.enabled.get()

### A convenient shorthand for common simple cases: `stage_sigs`

## Customizing cleanup via `stop`, `resume`, `pause`

In [ ]:
triggered_detector.exposure_time.get()